# Project Euler Problem 234: _Semidivisible numbers_

Link to original problem prompt: [https://projecteuler.net/problem=234](https://projecteuler.net/problem=234)

## Approach

Some might be tempted to loop through all integers not exceeding 999966663333 and check to see if each is semidivisible or not (by computing its _lower prime square root_ and _upper prime square root_). A faster approach relies on the fact that given two consecutive prime numbers, it is possible to work out all semidivisible numbers between them, and finally compute their sum.

### Semidivisible numbers between two consecutive prime numbers

Given that $p$ and $q$ are two consecutive prime numbers, we see that every integer $n$ such that $p < n < q$ have the same lower prime square root (specifically $p$) and also the same upper prime square root (specifically $q$). This means that if we divide our search space into sections between pairs of consecutive prime numbers, we can exhaust all semidivisible numbers in question.

The goal now is,  given $p$ and $q$, to calculate the sum of all semidivisible numbers between them. This is simply the sum of all numbers divisible by $p$ or $q$ but not by $p$ and $q$. Since $p$ and $q$ are prime numbers, if $n$ is divisible by both $p$ and $q$, it is also divisible by $p * q$. The question is now reduced to an algorithm that computes the sum of all multiples of a given integer $p$ inside a specific range.

\begin{align}
S & = p * n\ +\ p * (n + 1)\ +\ ...\ +\ p * m\\
& = p * (n\ +\ (n + 1)\ +\ ...\ +\ m)\\
& = p * [(1\ + 2\ +\ ...\ +\ m) - (1\ + 2\ +\ ...\ +\ n)]\\
& = p * [\frac{m * (m + 1)}{2} - \frac{n * (n + 1)}{2})]
\end{align}

With that, we can write the same function in Python:

In [1]:
def sum_divisibles(lower, upper, p):
    if lower >= limit:
        return 0

    refactored_lower = lower // p

    if upper > limit:
        upper = limit

    refactored_upper = upper // p if upper % p else (upper - 1) // p

    divisible_sum = p * (refactored_upper * (refactored_upper + 1)
                    - refactored_lower * (refactored_lower + 1)) // 2

    return divisible_sum

### Handling the upper limit

Notice in the above function that there is a variable called `limit`, this is the hard upper limit of semidivisible numbers that we are looking for. The limit actually needs to be handled carefully with the limit with which we generate our prime numbers. Specifically, there are edge-cases with disconnects between these two limits.

### Concurrency

As always, I'd like to spread the work across multiple threads just to gain some additional speedup:

In [2]:
def solve(limit, prime_input='primes.txt'):
    global running_sum

    def thread_worker():
        global running_sum

        while True:
            task = task_queue.get()
            if task is None:
                break

            p1, p2 = task
            p1_squared = p1 ** 2
            p2_squared = p2 ** 2

            p1_divisible_sum = sum_divisibles(p1_squared, p2_squared, p1)
            p2_divisible_sum = sum_divisibles(p1_squared, p2_squared, p2)
            p1_p2_divisible_sum = sum_divisibles(
                p1_squared, p2_squared, p1 * p2
            )
            semidivisible_sum = p1_divisible_sum + p2_divisible_sum\
                                - 2 * p1_p2_divisible_sum

            running_sum += semidivisible_sum

            task_queue.task_done()

    # reading in and cleaning saved prime numbers
    with open(prime_input, 'r') as f:
        lines = f.readlines()
    primes = list(map(lambda x: int(x[: -1]), lines))

    # setting up threading variables
    task_queue = Queue()
    threads = []
    n_threads = 4
    for i in range(n_threads):
        t = Thread(target=thread_worker)
        t.start()
        threads.append(t)

    # adding prime pairs to task queue
    for i in range(len(primes) - 1):
        task_queue.put((primes[i], primes[i + 1]))

    # blocking until all tasks are done
    task_queue.join()

    # poison-pill and stopping all threads
    for i in range(n_threads):
        task_queue.put(None)
    for t in threads:
        t.join()

    print('Final result:', running_sum)

## Conclusion

This is quitea straightforward problem once you have gained the above insight regarding pairs of consecutive primes. It actually took me a while to arrive at the insight myself.